In [ ]:
#Download data
import json
import urllib2

scdp_newjob_url='http://sensorcity-api.kxa.nl/newjob'
scdp_status_url='http://sensorcity-api.kxa.nl/status'
scdp_download_url='http://sensorcity-api.kxa.nl/download'


In [ ]:
import getpass

#scdp_user=getpass.getpass('User name: ')
scdp_user="kevin_hanze"
scdp_pw=getpass.getpass('User password: ')

#scdp_pw=""

In [ ]:
data=json.dumps(
{
    "username": scdp_user,
    "password": scdp_pw,
    "job": {
    "name": "newalarmtest",
    "datetime-start": "2016-sep-20 11:59:00",
    "datetime-end": "2016-sep-20 12:02:00",
    "type":"audio",
    "format":"wav",
    "sensors": ["45-AU-1","51-AU-1","93-AU-1","156-AU-1","167-AU-1","171-AU-1","208-AU-1","98-AU-1","107-AU-1","14-AU-1","20-AU-1","28-AU-1","29-AU-1","41-AU-1","42-AU-1","53-AU-1","59-AU-1","68-AU-1","140-AU-1","170-AU-1","201-AU-1","202-AU-1","213-AU-1","6-AU-1","158-AU-1","187-AU-1","111-AU-1","27-AU-1","85-AU-1","103-AU-1","149-AU-1",]
    }
  
})


'''

'''
clen = len(data)

req = urllib2.Request(scdp_newjob_url, data, {'Content-Type': 'application/json', 'Content-Length': clen})
f = urllib2.urlopen(req)
response = f.read()
f.close()
print response



In [ ]:
# Get status update use job-id obtained in previous step
import time

response_dict=json.loads(response)
jobid=response_dict[u'job'][u'job-id']

#jobid=1803612
#jobid=1803761
data=json.dumps(
{
    "username": scdp_user,
    "password": scdp_pw,
    "job-id": jobid
  }
)

clen = len(data)

waiting=True
    
while waiting:
        req = urllib2.Request(scdp_status_url, data, {'Content-Type': 'application/json', 'Content-Length': clen})
        f = urllib2.urlopen(req)
        status = f.read()
        f.close()
        status_dict=json.loads(status)
        if status_dict[u'job'][u'status'] == 'finished':
            waiting=False
        else:
            print status_dict[u'job'][u'status']
            time.sleep(5) #Sleep for x seconds between checks

print status_dict[u'job'][u'status']

In [ ]:
# Download the data

clen = len(data)

req = urllib2.Request(scdp_download_url, data, {'Content-Type': 'application/json', 'Content-Length': clen})

try:
    f = urllib2.urlopen(req)
    zipfile = open('rawwavfiles.zip','w')
    zipfile.write(f.read())
    zipfile.close()
    f.close()
    
#handle errors
except HTTPError, e:
    print "HTTP Error:", e.code, url
except URLError, e:
    print "URL Error:", e.reason, url

In [ ]:
#Delete old folder to be sure that no old files will be analysed
rm -rf sounddata

In [ ]:
from zipfile import ZipFile
#Extract data from zip
zip_ref = ZipFile("rawwavfiles.zip", 'r')
zip_ref.extractall("sounddata")
zip_ref.close()

In [ ]:
#Configure sound analysis
%run ~/soundannotatordemo/soundannotatordemo/projects/simple/soundAnnotator.py  -c --noofscales 100 --inputrate 44100 --outdir ~/data --decimation 2

In [ ]:
#Perform sound analysis on each file in sounddata folder

%run ~/soundannotatordemo/soundannotatordemo/projects/simple/soundAnnotator.py --PTNE  --ptnblockwidth 0.1 --noofscales 100 --inputrate 96000 --outdir ~/data --decimation 2 --wav ~/SensorCity/sounddata/


In [1]:
import os

# d='E:\Users\Kevin\Google Drive\MasterSSE\Advanced Data Analysis'
# hdfdirs = [os.path.join(d,o) for o in os.listdir(d) if os.path.isdir(os.path.join(d,o))]
# hdfdirs[0]

# #Find folder which contains the ptne data
# for n,line in enumerate(hdfdirs):
#     #print hdfdirs[n]
#     if (os.path.isdir(os.path.join(hdfdirs[n],'ptne/')) == 1):
#         print "PTNE FOLDER FOUND: " + hdfdirs[n] + "\n"
#         ptnefolder = os.path.join(hdfdirs[n],'ptne/hdf5')
        
# d=ptnefolder
# currentdatefolder = [os.path.join(d,o) for o in os.listdir(d) if os.path.isdir(os.path.join(d,o))]
# ptnefolder = os.path.join(ptnefolder,currentdatefolder[0])

#ptnefolder = "E:\Users\Kevin\Google Drive\MasterSSE\Advanced Data Analysis\hdf5files\GIT_cc14a7a227c856aee6e0b541b97a77aabb8d3307-2016-10-15-13-56\ptne\hdf5\"

ptnefolder = 'E:\\Users\\Kevin\\Google Drive\\MasterSSE\\Advanced Data Analysis\\hdf5files\\GIT_cc14a7a227c856aee6e0b541b97a77aabb8d3307-2016-10-15-13-56\\ptne\\hdf5\\2016-10-15'

In [3]:
# Analyze HDF5 files
%matplotlib
import h5py
import numpy as np
import os
import matplotlib.pyplot as plt
ptnefolder = 'E:\\Users\\Kevin\\Google Drive\\MasterSSE\\Advanced Data Analysis\\hdf5files\\GIT_cc14a7a227c856aee6e0b541b97a77aabb8d3307-2016-10-15-13-56\\ptne\\hdf5\\2016-10-15'

for filename in os.listdir(ptnefolder):
    #print filename
    filepath = os.path.join(ptnefolder,filename)
    
    with h5py.File(filepath,'r') as hf:
        datae = hf.get('energy')
        np_datae = np.array(datae)
        
        datan = hf.get('noise')
        np_datan = np.array(datan)
    
        datap = hf.get('pulse')
        np_datap = np.array(datap)

        datat = hf.get('tone')
        np_datat = np.array(datat)
        
        if(True):
            #print('shape:',np_datae.shape)
            F, (ax1, ax2, ax3, ax4) = plt.subplots(4, sharex=True)
            ax1.imshow(np_datae[:18,270:1350], interpolation='nearest',aspect='auto')
            ax1.set_title('E')
            ax2.imshow(np_datan[:18,270:1350], interpolation='nearest',aspect='auto')
            ax2.set_title('N')
            ax3.imshow(np_datap[:18,270:1350], interpolation='nearest', aspect='auto')
            ax3.set_title('P')
            ax4.imshow(np_datat[:18,270:1350], interpolation='nearest', aspect='auto')
            ax4.set_title('T')
            F.canvas.set_window_title(filename)
            F.tight_layout()
            
            print filename[:-5] + ':   ' + str(np.average(np_datae[5,270:1350]))
            
            #Uncomment to save files
            #files = filename[:-5] + '.jpeg'
            #F.savefig(files)
        
        if(True):
            test = 0
        
plt.show()

Using matplotlib backend: Qt5Agg
103:   58.8775
107:   60.5376
111:   59.8553
14:   56.8687
140:   60.3894
149:   62.2716
156:   62.8035
158:   37.4962
167:   39.845
170:   55.4812
171:   61.2673
187:   47.4162
20:   57.3992
201:   68.4814
202:   59.625
208:   43.4782
213:   58.9486
27:   44.3194
28:   57.0138
29:   48.738
41:   59.4212
42:   62.1834
45:   -inf
51:   57.9214
53:   57.5754
59:   64.6204
6:   58.741
68:   61.1767
85:   61.2654
93:   57.9285
98:   61.6538


# List of sensors and their status

| Sensor        | Status           | Notes  |
| :-------------: |:-------------:| :-----:|
|    6   | Working |  |
|    14   | Working |  |
|     20  | Working |  |
|   28    | Working |  |
|    41   | Working |  |
|   42    |Working  |  |
|     51  |  Working|  |
|    53   | Working |  |
|    59   | Working |  |
|   68    |Working  |  |
|   85    | Working |  |
|    93   | Working |  |
|    98   |Working  |  |
|    103   | Working |  |
|   107    | Working |  (has a high tone throughout the entire sample)|
|   111    | Working |  |
|   140    | Working |  |
|   149    | Working |  |
|   156    | Working |  |
|   170    | Working |  |
|     171  | Working |  |
|    201   | Working |  |
|   202    |  Working| Static noise throughout sample |
|   213    |Working  |  cannot pick up siren|
|    27   | Not Working |  |
|    29   | Not Working |  |
|   45    | Not Working |  |
|    158   | Not Working |  |
|     167  | Not Working |  |
|    187   | Not Working |  |
|   208    |Not Working  | Vibrations/static |

123
